In [ ]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn. optim

In [ ]:
#create workers

bob = sy.VirtualWorker(hook, id="bob") #data owner
alice = sy.VirtualWorker(hook, id="alice") #data owner
secure_worker = sy.VirtualWorker(hook, id="secure_worker") #trusted third party/ trusted aggregator

In [ ]:
#inform workers tht other workers exist
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([bob, alice])

In [ ]:
# a toy dataset
data = th.tensor([[0,0], [0,1], [1,0], [1,1.]], requires_grad=True)
target = th.tensor([[0], [0], [1], [1.]], requires_grad=True)

In [ ]:
#get pointers to training data on each worker by sending some training data to bob and alice
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

alices_data = data[2:].send(alice)
alices_target = data[2:].send(alice)

In [ ]:
#Initialize a toy model
model = nn.Linear(2,1)

In [ ]:
for round_iter in range[0,10]:
#since there are 2 model copies, each will have its own optimizers
    bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
    alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

    #train the model on bob's device and alice's

    for i in range(10):
    bobs_opt.zero_grad()
        bobs_pred = bobs_model(bobs_data)
        bobs_loss = ((bobs_pred-bobs_target)**2).sum()
        bobs_loss.backward()
        bobs_opt.step()
        bobs_loss = bobs_loss.get().data
        bobs_loss #tensor(0.0054)

        alices_opt.zero_grad()
        alices_pred = alices_model(alices_data)
        alices_loss = ((alices_pred-alices_target)**2).sum()
        alices_loss.backward()
        alices_opt.step()
        alices_loss = alices_loss.get().data
        alices_loss #tensor(0.0015) not exact value!

    alices_model.move(secure_worker)
    bobs_model.move(secure_worker)

    model.weight.data.set_(((alices_model.weights.data + bobs_model.weights.data) / 2).get())
    model.bias.data.set_(((alices_model.bias.data+ bobs_model.bias.data) / 2).get())

    print("Bob:" + str(bobs_loss) + " Alice: ", str(alices_loss))
    


In [ ]:
#ADDITIVE SECRET SHARING

In [ ]:
import random

In [ ]:
Q = 23704657892341988765345
x = 5

In [ ]:
def encrypt(x, n_shares=3)
    shares = list()
    for i in range(n_shares-1):
        shares.append(random.randint(0,Q))

    final_share = Q - (sum(shares)%Q) + x
    shares.append(final_share)

    #DEMO sum(shares) % Q #output is 5 which is x
    
    return tuple(shares)
     

In [ ]:
def decrypt(shares):
    return sum(shares) % Q

In [ ]:
def add(a,b):
    c = list()
    assert(len(a)==len(b))
    for i in range(len(a)):
        c.append((a[i]+b[i]) % Q)
    return tuple(c)

In [ ]:
#DEMO
decrypt(add(encrypt(5), encrypt(10))) #output is 15

In [ ]:
#MORE EXPLICITLY EXPLAINED

x = encrypt(5)
x #output is tuple of 3 huge numbers i.e shares ||| Q

y=encrypt(10)
y #output is tuple of 3 huge numbers i.e shares ||| Q

z = add(x,y)
z #output is tuple of 3 huge numbers i.e sum of shares ||| Q

decrypt(z) #output is 15

In [ ]:
#FIXED PRECISION ENCODING

#additive secret sharing is applicable only to integers. But gradients in federated learning are mostly decimals. Hence we use
#this technique!

In [ ]:
BASE = 10 #we can use 2 for binary...here we r using base 1-
PRECISION = 4
Q = 23704657892341988765345

In [ ]:
def encode(x_dec): #convert dec to int
    return int(x_dec * (BASE ** PRECISION)) % Q

#DEMO encode(0.5) #output os 5000
#DEMO encode(-0.5) #output is a large number lyk Q

In [ ]:
def decode(x_fixedprecision):
    return (x_fixedprecision if x_fixedprecision <= Q/2 else x_fixedprecision - Q ) / BASE ** PRECISION

#DEMO decode(5000) #output is 0.5
#DEMO decode(num_lyk_Q) #output is -0.5

In [ ]:
#SECRET SHARING + FIXED PRECISION WITH PYSYFT

In [ ]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [ ]:
x = th.tensor([1,2,3,4,5])

In [ ]:
x = x.share(bob, alice, secure_worker) #divides x into multiple diffnt additive secret shares. Any num of workers can b gvn.

#it sends the shares to the workers in such a way that we have pointers to the data

In [ ]:
bob._objects #output is a set of 5 large random numbers

#same for alice and secure_worker as well.

y = x + x

bob._objects #output is 2 sets of 5 shares each.

In [ ]:
y.get() #will decrypt the tensor and return the original sum value of x+x...i.e. [2,4,6,8,10]

In [ ]:
#To suit for decimal values
x = th.tensor([0.1, 0.2, 0.3, 0.4, 0.5]).fix_prec.(share(bob, alice, secure_worker)
                                                   
#below code is commented cuz we have included that in definition of x above.                                                   
"""
x = x.fix_prec
x #output is [100,200,300,400,500]

x = x.float_prec
"""